<a rel="license" href="http://creativecommons.org/licenses/by-nc-sa/4.0/"><img alt="Creative Commons License" align="left" src="https://i.creativecommons.org/l/by-nc-sa/4.0/80x15.png" /></a>&nbsp;| [Emmanuel Rachelson](https://personnel.isae-supaero.fr/emmanuel-rachelson?lang=en) | <a href="https://github.com/erachelson/RLtuto">https://github.com/erachelson/RLtuto</a>

<div style="font-size:22pt; line-height:25pt; font-weight:bold; text-align:center;">A walk in the garden of foundational ingredients, <br> theorems and algorithms in reinforcement learning</div>

Welcome to the RL restaurant! On the menu:
- Central intuitions for RL (section 1)
- Setting the stage (sections 2, 3, 4)
- Finally some learning (sections 5, 6)

My intention in this tutorial:
- a compendium of definitions, theorems and algorithms that you can re-use,
- minimal text, lots of room for discussion,
- rigorous mathematical understanding and algorithmic notions at the same time.

# Intuitions

## A medical prescription example

<center><img src="img/patient-doctor.png" style="height: 200px;"></center>

## Patient variables

<center>
<img src="img/patient_file.png" style="height: 100px;"> </img> <br>
Patient state now: $S_0$  <br>
Future states: $S_t$
</center>

$S_t$ random variable  
$\mathcal{S}$ patient description space, state space

## Prescription

<center>
<img src="img/prescription.png" style="height: 100px;"> </img> <br>
Prescription: $\left( A_t \right)_{t\in\mathbb{N}} = (A_0, A_1, A_2, ...)$
</center>

$A_t$ random variable  
$\mathcal{A}$ prescription space, action space

## Patient evolution


<center>
<img src="img/patient_evolution.png" style="height: 100px;"> </img> <br>
    $\mathbb{P}(S_t)$?
</center>

$\left( S_t \right)_{t\in\mathbb{N}}$ random process

## Physician's goal

<center><img src="img/patient_happy.png" style="height: 100px;"> </img> <br></center>

$$J \left( \left(S_t\right)_{t\in \mathbb{N}}, \left( A_t \right)_{t\in \mathbb{N}} \right)?$$

Make (keep) patient healthy, from $S_0$, until time horizon.  
$J \left( \left(S_t\right)_{t\in \mathbb{N}}, \left( A_t \right)_{t\in \mathbb{N}} \right)$: how good is a trajectory in the joint $S\times A$ space?

## Wrap-up

- Patient state $S_t$, random variable with values in $\mathcal{S}$,
- Physician instruction $A_t$, random variable with values in $\mathcal{A}$,
- Prescription $\left( A_t \right)_{t\in\mathbb{N}}$, sequence of random variables, random process  
- Patient's evolution $\mathbb{P}(S_t)$,  
- Patient's state trajectory $\left( S_t \right)_{t\in\mathbb{N}}$, random process, 
- Patient's full trajectory $\left( S_t, A_t \right)_{t\in\mathbb{N}}$, random process, 
- Value of a trajectory $J \left( \left(S_t, A_t \right)_{t\in \mathbb{N}} \right)$.  

It seems reasonable that the physician's recommendation $\mathbb{P}(A_t)$ at step $t$ be dependent on previously observed states $\left(S_0, \ldots, S_t\right)$ and recommended treatments $\left(A_0, \ldots, A_{t-1}\right)$.

## Common misconception

You will often see the following type of drawing, along with a sentence like "RL is concerned with the problem on an agent performing actions to control an environment". 

<center><img src="img/misconception.png" style="height: 300px;"></img></center>

Not false, but prone to (anthropomorphic) misconceptions:  
No separation between a *state of the agent* and a *state of the environment*.

Less shiny but more accurate drawing:  

<center><img src="img/dynamic.png" style="height: 300px;"></img></center>

A *System to control*, described through its observed *state*, controlled by the application of actions issued from a *policy* or *control law*. The process of *learning* this policy is what RL is concerned with.

# Ingredients (definitions)

## Markov decision processes

<div class="alert alert-info"><b>Definition: Markov Decision Process (MDP)</b><br>
A Markov Decision Process is given by:
<ul>
<li> A measurable set of states $\mathcal{S}$
<li> A measurable set of actions $\mathcal{A}$
<li> A (Markovian) transition model $\mathbb{P}\left(S_{t+1} | S_t, A_t \right)$, noted $p(s'|s,a)$
<li> A reward model $\mathbb{P}\left( R_t | S_t, A_t, S_{t+1} \right)$, noted $r(s,a)$ or $r(s,a,s')$
<li> A set of discrete decision epochs $\mathcal{T}=\{0,1,\ldots,h\}$
</ul>
</div>

**Examples**  
- video games,
- robots,
- agro-ecosystems,
- biological systems,
- financial marketplaces,
- communication networks,
- supply-chain systems, etc.

In short: any fully-observable "$s_{t+1} = f(s_t,a_t) + noise$" dynamical system.

Today's tutorial: $h=+\infty$.

In [ ]:
# The "frozen lake" toy MDP
import gymnasium as gym
import gymnasium.envs.toy_text.frozen_lake as fl
import matplotlib.pyplot as plt
env = gym.make('FrozenLake-v1', render_mode="rgb_array")
env.reset()
plt.imshow(env.render());

[Link to the Gymnasium website](https://gymnasium.farama.org/)

In [ ]:
# State and action space
print("State space: ", env.observation_space)
print("Action space:", env.action_space)

In [ ]:
import gymnasium.envs.toy_text.frozen_lake as fl
actions = {fl.LEFT: '\u2190', fl.DOWN: '\u2193', fl.RIGHT: '\u2192', fl.UP: '\u2191'}
print(actions)

In [ ]:
# Transition model
s=2
a=1
print("Example of transition: (s=", s, 
      ",a=", actions[a], 
      ") \u27FF s=", env.unwrapped.P[s][a][0][1], 
      " with proba ", env.unwrapped.P[s][a][0][0], 
      ".", sep='')
print("All transitions from (s=", s, 
      ",a=", actions[a], 
      "):", sep='')
print(env.unwrapped.P[s][a])

In [ ]:
# Reset the environment
s, prob = env.reset()
print("Initial state:", s)

In [ ]:
# Take an action in the MDP
a=2
print("Old state:", s)
s, r, done, truncated, info = env.step(a)
print("Action:   ", a, " (", actions[a], ")", sep="")
print("New state:", s)
print("Reward:   ", r)
print("Game over:", done)
print("Truncated:", truncated)
plt.imshow(env.render());

## Policies

Formally, how does one write the behavior of an agent?
This behavior specifies how to choose actions at each time step:
$$A_t \sim \pi_t.$$

Let $\Delta_\mathcal{A}$ be the set of probability measures on the action space $\mathcal{A}$. Then the law $\pi_t$ of $A_t$ belongs to $\Delta_\mathcal{A}$.  
$\pi_t$ is called the **decision rule** at step $t$, it is a distribution over the action space $\mathcal{A}$.  
The collection $\pi = \left(\pi_t \right)_{t\in \mathcal{T}}$ is called a **policy**.

<div class="alert alert-info"><b>Definition: policy $\pi$</b><br>
A policy $\pi$ is a sequence of decision rules $\pi_t$: $\pi = \{\pi_t\}_{t\in \mathcal{T}}$, with $\pi_t \in \Delta_\mathcal{A}$.
</div>

Remark: $\pi_t$ might depend on the full history of $S_0,A_0,\ldots,S_{t-1},A_{t-1},S_t$.

In [ ]:
# A deterministic, stationary, state-dependent policy which always moves right
import numpy as np
pi = np.ones(env.observation_space.n)
print("A deterministic, stationary, state-dependent policy:", pi)

In [ ]:
# A randomly-drawn deterministic, stationary, state-dependent policy
max_action = env.action_space.n - 1
pi = np.round(max_action * np.random.random((env.observation_space.n)))
print("A randomly-drawn deterministic, stationary, state-dependent policy:", pi)

In [ ]:
def print_policy(pi):
    for row in range(env.unwrapped.nrow):
        for col in range(env.unwrapped.ncol):
            s = row*env.unwrapped.ncol+col
            print(actions[pi[s]], end='')
        print()
    return

In [ ]:
print_policy(pi)

## Values 

<div class="alert alert-info">

Definition: the **state return** random variable:
$$G^\pi(s) = \sum\limits_{t = 0}^\infty \gamma^t R_t \quad \Bigg| \quad \begin{array}{l}S_0 = s,\\ A_t \sim \pi_t,\\ S_{t+1}\sim p(\cdot|S_t,A_t),\\R_t = r(S_t,A_t,S_{t+1}).\end{array}$$
</div>

<div class="alert alert-info"><b>Definition: value function $v^\pi$ of a policy $\pi$ under a $\gamma$-discounted criterion</b><br>
$$v^\pi : \left\{\begin{array}{ccl}
\mathcal{S} & \rightarrow & \mathbb{R}\\
s & \mapsto & v^\pi(s)=\mathbb{E}\left( \sum\limits_{t = 0}^\infty \gamma^t R_t \bigg| S_0 = s, \pi \right)\end{array}\right. $$
</div>

<div class="alert alert-info">

Definition: the **state-action return** random variable:
$$G^\pi(s,a) = \sum\limits_{t = 0}^\infty \gamma^t R_t \quad \Bigg| \quad \begin{array}{l}S_0 = s, A_0=a\\ A_t \sim \pi_t\textrm{ for }t\geq 1,\\ S_{t+1}\sim p(\cdot|S_t,A_t),\\R_t = r(S_t,A_t,S_{t+1}).\end{array}$$
</div>

<div class="alert alert-info"><b>Definition: state-action value function $q^\pi$</b><br>
$$q^\pi(s,a) = \mathbb{E}\left( \sum\limits_{t=0}^\infty \gamma^t r\left(S_t, A_t, S_{t+1}\right) \bigg| S_0 = s, A_0=a, \pi \right)$$
</div>

In [ ]:
# Monte Carlo evaluations
import numpy as np
pi0 = 0*np.ones(env.observation_space.n)
pi1 = 1*np.ones(env.observation_space.n)
pi2 = 2*np.ones(env.observation_space.n)
pi3 = 3*np.ones(env.observation_space.n)
pi4 = np.round(3*np.random.random((env.observation_space.n)))
pi5 = np.array([2, 2, 1, 0, 2, 2, 1, 0, 2, 2, 1, 0, 2, 2, 2, 2])
policies = [pi0, pi1, pi2, pi3, pi4, pi5]
rollouts = 100
horizon = 200
gamma = .99
v0 = 0

for pi in policies:
    for ep in range(rollouts):
        s, _ = env.reset()
        G = 0
        for t in range(horizon):
            a = pi[s]
            s, r, done, _, _ = env.step(a)
            G += gamma**t * r
            if done:
                break
        v0 += G
    v0 = v0/rollouts
    print_policy(pi)
    print("    v^pi(s_0) =", v0)

## Optimality

<div class="alert alert-info">
    
**Definition: optimal policy $\pi^*$**  
$\pi^*$ is said to be optimal iff $\pi^* \in \arg\max\limits_{\pi} v^\pi$.  
    
A policy is optimal if it **dominates** over any other policy in every state:
$$\pi^* \textrm{ is optimal}\Leftrightarrow \forall s\in \mathcal{S}, \ \forall \pi, \ v^{\pi^*}(s) \geq v^\pi(s)$$
</div>

<div class="alert alert-info">
    
**Definition: optimal value function**  
$v^*$ is the unique value function of any optimal policy.  
$q^*$ is the unique state-action value function of any optimal policy.
</div>

# Optimal policies

<div class="alert alert-success"><b>Theorem: family of optimal policies</b><br>
There always exists at least one optimal stationary, deterministic, memoryless policy.</div>

Consequence: much easier to search for than non-stationary, stochastic, history-dependent policies.

<div class="alert alert-success">

**Theorem: the policy optimization problem:**  
Provided $\rho_0$ has non-zero probability mass on all states, an optimal policy is a solution to $\max_\pi J(\pi) = \mathbb{E}_{s_0\sim \rho_0}[v^\pi(s_0)]$.
</div>

Consequence: single scalar optimization problem, rather than $|\mathcal{S}|$ coupled optimization problems.

<div class="alert alert-info">
    
**Definition: greediness operator**  
For deterministic policies:
$$\pi \in \mathcal{G} q, \Leftrightarrow \pi(s) \in \arg\max_{a\in \mathcal{A}} q(s,a)$$

This can be extended to stochastic policies:
$$\pi \in \mathcal{G} q, \Leftrightarrow \pi(s) \in \arg\max_{\delta \in \Delta_\mathcal{A}} \mathbb{E}_{a\sim\delta} \left[q(s,a)\right]$$
</div>

<div class="alert alert-success">
    
**Theorem: Optimal greedy policy**  
$\pi \in \mathcal{G}q^*$ is an optimal (deterministic or stochastic) policy.  
</div>

Consequence: finding $q^*$ provides $\pi^*$ if one can solve this $\max_a$ easily.

In [ ]:
# Greedy policy from a random q-function
import numpy as np
q = np.random.random((env.observation_space.n, env.action_space.n))

def greedyQpolicy(Q):
    pi = np.zeros((env.observation_space.n),dtype=int)
    for s in range(env.observation_space.n):
        pi[s] = np.argmax(Q[s,:])
    return pi

print("q-function:\n", q)
print("greedy deterministic policy:\n",greedyQpolicy(q))

# Characterizing value functions: the Bellman equations

<div class="alert alert-success">
    
**Theorem: the evaluation equation**  
$v^\pi$ obeys the (full rank) linear system of equations:
$$v^\pi\left(s\right) = r(s,\pi(s)) + \gamma \mathbb{E}_{s'\sim p(s'|s,\pi(s))} \left[ v^\pi(s') \right], \forall s\in \mathcal{S}$$

$q^\pi$ obeys the (full rank) linear system of equations:
$$q^\pi\left(s,a\right) = r(s,a) + \gamma \mathbb{E}_{s'\sim p(s'|s,a)} \left[ q^\pi(s',\pi(s')) \right], \forall s, a\in \mathcal{S}\times\mathcal{A}$$
</div>

Comment: this is a dynamic programming equation. The value $v^\pi(s)$ of the trajectory starting in $s$ is the sum of the first step's reward $r(s,\pi(s))$ and the discounted value from the outcome state $v^\pi(s')$.

<div class="alert alert-info">
    
**Definition: Bellman evaluation operator $T^\pi$**
$$(T^\pi v)\left(s\right) = r(s,\pi(s)) + \gamma \mathbb{E}_{s'\sim p(s'|s,\pi(s))} \left[ v(s') \right]$$
$$(T^\pi q)\left(s,a\right) = r(s,a) + \gamma \mathbb{E}_{s'\sim p(s'|s,a)} \left[ q(s',\pi(s')) \right]$$
</div>

The evaluation equation:  
$v^\pi$ is the unique solution to $v=T^\pi v$,  
$q^\pi$ is the unique solution to $q = T^\pi q$.

<div class="alert alert-success">
    
**Theorem: the optimality equation**  
$$v^*(s) = \max\limits_{a\in \mathcal{A}} \left[ r(s,a) + \gamma \mathbb{E}_{s'\sim p(s'|s,a)} v^*(s') \right]$$
$$q^*(s,a) = r(s,a) + \gamma \mathbb{E}_{s'\sim p(s'|s,a)} \left[ \max_{a'\in \mathcal{A}} q^*(s',a') \right]$$
</div>

Comment: this is a dynamic programming equation again. The largest expected return $v^*(s)$ is the largest possible sum of the one-step reward $r(s,a)$ and the optimal value from to outcome state $v^*(s')$.

<div class="alert alert-info">
    
**Definition: Bellman optimality operator**  
$$(T^*v)(s) = \max\limits_{a\in \mathcal{A}} \left[ r(s,a) + \gamma \mathbb{E}_{s'\sim p(s'|s,a)} v(s') \right]$$
$$(T^*q)(s,a) = r(s,a) + \gamma \mathbb{E}_{s'\sim p(s'|s,a)} \left[ \max_{a'\in \mathcal{A}} q(s',a') \right]$$
</div>

<div class="alert alert-success">
    
**Theorem: $T^*$ and $T^\pi$ are $L_\infty$-contraction mappings**  

</div>

Consequence: unicity of a solution to the optimality equation.  
$v^*$ is the unique solution to $v=T^*v$,  
$q^*$ is the unique solution to $q=T^*q$.

# Dynamic programming algorithms

$v^\pi$ is the limit of the sequence $v_{n+1} = T^\pi v_n$, $q^\pi$ is the limit of the sequence $q_{n+1} = T^\pi q_n$,  
$v^*$ is the limit of the sequence $v_{n+1} = T^* v_n$, $q^*$ is the limit of the sequence $q_{n+1} = T^* q_n$.

<div class="alert alert-warning">

**Algorithm: value iteration**  
The algorithm that computes the sequence $Q_{n+1} = T^* Q_n$ for a finite number of iterations is called **value iteration**.
</div>

In [ ]:
# Value iteration
# Input: nb iterations, q0
# Output: q*

import gymnasium as gym
import gymnasium.envs.toy_text.frozen_lake as fl
import numpy as np
# use render_mode="human" to open the game window
env = gym.make('FrozenLake-v1', render_mode="rgb_array")

nb_iter = 20
gamma = 0.9
q = np.zeros((env.observation_space.n, env.action_space.n))
qopt_sequence = [q]
for i in range(nb_iter):
    qnew = np.zeros((env.observation_space.n, env.action_space.n))
    for x in range(env.observation_space.n):
        for a in range(env.action_space.n):
            outcomes = env.unwrapped.P[x][a]
            for o in outcomes:
                p = o[0]
                y = o[1]
                r = o[2]
                qnew[x,a] += p * (r + gamma*np.max(q[y,:]) )
    q = qnew
    qopt_sequence.append(q)

In [ ]:
print_policy(greedyQpolicy(q))

In [ ]:
# Plotting the convergence of the sequence
import matplotlib.pyplot as plt
%matplotlib inline

residuals = []
for i in range(1, len(qopt_sequence)):
    residuals.append(np.max(np.abs(qopt_sequence[i]-qopt_sequence[i-1])))

plt.plot(residuals)
plt.figure()
plt.semilogy(residuals);

Remark:  
$T^*q = T^\pi q$ with $\pi \in \mathcal{G} q$.

<div class="alert alert-warning">

**Algorithm: value iteration (reformulated)**  
$$\pi_n \in \mathcal{G} q_n, \ q_{n+1} = T^{\pi_n} q_n$$
</div>

VI converges to $q^*$.

<div class="alert alert-warning">

**Algorithm: policy iteration**  
$$\pi_n \in \mathcal{G} q_n, \ q_{n+1} \textrm{ is a solution to } q=T^{\pi_n} q$$
</div>

<div class="alert alert-warning">

**Algorithm: modified policy iteration**  
$$\pi_n \in \mathcal{G} q_n, \ q_{n+1} = {(T^{\pi_n})}^m q_n$$
</div>

VI is MPI with $m=1$.  
PI is MPI with $m=\infty$.

All converge to $q^*$ and $\pi^*$ (whatever their initialization).

<div class="alert alert-success">

**Theorem: time complexity of modified policy iteration**  

TODO
</div>

# Approximate dynamic programming algorithms

<div class="alert alert-warning">
    
**Approximate value iteration** is the algorithm that computes the sequence $Q_{n+1} = \mathbb{A} T^* Q_n$, where $\mathbb{A}$ is an approximation procedure.
</div>

<div class="alert alert-warning">
    
**Approximate value iteration (reformulated)**
$$\pi_n \in \mathcal{G} q_n, \ q_{n+1} = \mathbb{A} T^{\pi_n} q_n$$
</div>

<div class="alert alert-success">
    
**Theorem: asymptotical behavior of AVI**  

If $\| f-\mathbb{A}f \|_\infty \leq \epsilon, \forall f \in \mathbb{R}^{SA}$,

$$\exists N\in \mathbb{N}, \textrm{such that }\| Q^* - Q_n \|_\infty \leq \frac{\epsilon}{1-\gamma}, \forall n\leq N.$$

Let $\pi_n$ be the greedy policy with respect to $Q_n$,
$$\|Q^*-Q^{\pi_n}\|_\infty \leq \frac{2\gamma\epsilon}{(1-\gamma)^2}, \forall n \leq N.$$

</div>
Warning: AVI does not (necessarily) converge!  
But it reaches policies whose values are close to optimal.

Info: similar bounds in weighted L2 norm (more suited to learning).

TODO: add bounds in L2 norm with concentration inequalities.

<div class="alert alert-warning">
    
**Approximate modified policy iteration**
$$\pi_n \in \mathcal{G} q_n, \ q_{n+1} = \mathbb{A} {(T^{\pi_n})}^m q_n$$
</div>

TODO: concentration inequalities for AMPI.

Example of $\mathbb{A}$: a procedure that minimizes $L(\theta) = \mathbb{E}_{x,y} [ (f_\theta(x) - y)^2 ]$, or more generally $L(\theta) = \mathbb{E}_{x,y} [ \ell(f_\theta(x), y) ]$.

Consequence: AVI (or AMPI) can be cast as a sequence of supervised learning problems if we can have samples of $(T{^{\pi})}^m q$.

# Learning for policy evaluation

The **return** random variable:
$$G^\pi(s,a) = \sum\limits_{t = 0}^\infty \gamma^t R_t \quad \Bigg| \quad \begin{array}{l}S_0 = s, A_0=a\\ A_t \sim \pi_t\textrm{ for }t\geq 1,\\ S_{t+1}\sim p(\cdot|S_t,A_t),\\R_t = r(S_t,A_t,S_{t+1}).\end{array}$$
$$q^\pi(s,a) = \mathbb{E}[G^\pi(s,a)]$$

$q^\pi(s,a)$ is a minimizer of $L(q) = \mathbb{E} \left[ \left( q - G^\pi(s,a) \right)^2 \right]$.   
Getting samples from $G^\pi(s,a)$ = Monte Carlo rollouts.

<div class="alert alert-warning">
    
**Policy evaluation as stochastic approximation**  
If we can obtain independent realizations $g^\pi(s,a)$ of $G^\pi(s,a)$ in all $s,a$, we can perform stochastic approximation updates of $q$ under the form:
$$q(s,a) \leftarrow q(s,a) + \alpha \left(g^\pi(s,a) - q(s,a)\right).$$
Then $q$ converges to $q^\pi$.
</div>

<div class="alert alert-warning">
    
**Policy evaluation as stochastic gradient descent (1/2)**  
If we can access a set $\left\{g^\pi_i(s,a)\right\}_{i\in [1,N]}$ of $N$ independently drawn realizations of $G^\pi(s,a)$ in all $s,a$, we can perform stochastic gradient descent updates of $q$ in each $s,a$ independently, under the form:
$$q(s,a) \leftarrow q(s,a) + \alpha \frac{1}{N} \sum_{i=1}^N \left[g^\pi_i(s,a) - q(s,a)\right].$$
Then $q$ converges to $q^\pi$.
</div>

The formulation above requires updates in each $(s,a)$ separately.  
Risk minimization problem with parametric $q_\theta$:
$$L(\theta) = \mathbb{E}_{\substack{(s,a)\sim \rho\\ g^\pi(s,a)\sim G^\pi(s,a)}} \left[ \left( q_\theta(s,a) - g^\pi(s,a) \right)^2\right].$$

<div class="alert alert-warning">
    
**Policy evaluation as stochastic gradient descent (2/2)**  
Provided we have a set $\{(s_i,a_i,g^\pi_i\}_{i\in [1,N]}$ where the $(s_i,a_i)$ are independent realizations of $(s,a)$ drawn according to $\rho$ and $g^\pi_i$ are independent realizations of $G^\pi(s_i,a_i)$, then we can perform stochastic gradient descent updates of the parameters $\theta$ of a parametric function $q_\theta$, under the form:
$$\theta \leftarrow \theta + \alpha \frac{1}{N} \sum_{i=1}^N \left[g^\pi_i - q_\theta(s_i,a_i) \right] \nabla_\theta q_\theta(s_i,a_i).$$
Then $q_\theta$ converges to some approximation of $q^\pi$ on the support of $\rho$.
</div>

<div class="alert alert-info">

Definition: the **$m$-step bootstrapped return** random variable:
$$G^\pi_m(s,a,q) = \sum\limits_{t = 0}^{m-1} \gamma^t R_t + \gamma^m q(S_m, A_m) \quad \Bigg| \quad \begin{array}{l}S_0 = s, A_0=a\\ A_t \sim \pi(S_t)\textrm{ for }t>0,\\ S_{t+1}\sim p(\cdot|S_t,A_t),\\R_t = r(S_t,A_t,S_{t+1}).\end{array}$$
</div>
$${(T^\pi)}^m q = \mathbb{E}[G^\pi_m(s,a,q)]$$

<div class="alert alert-info">

Definition: the **(1-step) bootstrapped return** random variable:
$$G^\pi_1(s,a,q) = R_0 + \gamma q(S_1, A_1) \quad \Bigg| \quad \begin{array}{l}S_0 = s, A_0=a\\ A_1 \sim \pi(S_1),\\ S_{1}\sim p(\cdot|S_0,A_0),\\ R_0 = r(S_0,A_0,S_{1}).\end{array}$$
</div>
$${T^\pi} q = \mathbb{E}[G^\pi_1(s,a,q)]$$

Consequence: getting samples from $G^\pi_1(s,a,q)$ enables learning ${T^\pi} q$.  
Getting samples from $G^\pi_1(s,a,q)$ = getting samples $r\sim R_0$, $s'\sim p(\cdot|s,a)$ and $a'\sim \pi(s')$, then summing $r + \gamma q(s',a')$.  
Sampling the bootstrapped return: dynamic programming as a sequence of supervised learning problems.

<div class="alert alert-warning">

**Approximate dynamic programming as a sequence of risk minimization problems.**  
Approximate dynamic programming can be cast as finding the sequence of functions $q(s,a;\theta_n)$ defined by $\theta_{n+1} \in \arg\min_{\theta} L_n(\theta)$, with
$$L_n(\theta) = \frac{1}{2} \mathbb{E}_{(s,a) \sim \rho}\left[ \left( q(s,a;\theta) - G^\pi_1(s,a,q_n) \right)^2 \right].$$

If this risk is differentiable, provided one can draw a mini-batch of independently and identically drawn samples $\left\{\left(s_i,a_i,r_i,s'_i\right)\right\}_{i\in [1,B]}$ (either by sampling from a larger training set, or directly from the system to control), with $(s,a) \sim \rho(\cdot)$ and $s',a' \sim p(s' | s,a)\pi(a'|s')$, then one can derive a stochastic gradient descent learning procedure and iteratively learn $\theta_{n+1}$ as the limit of the sequence $\theta_{k+1} \leftarrow \theta_{k} + \alpha_k d_n(\theta_{k})$ with 
$$d_n(\theta) = \frac{1}{B} \sum_{i=1}^B \left[ \left( r_i + \gamma q(s_i',a';\theta_{n}) - q(s_i,a_i;\theta) \right) \nabla_\theta q(s_i,a_i;\theta) \right].$$
</div>

Taking $B=1$ and $(s,a)$ as the current state and action in the previous algorithm yields the historical TD(0) algorithm.

# Learning optimal value functions

<div class="alert alert-warning">

**Approximate value iteration as a sequence of risk minimization problems**  
$$\pi_n \in \mathcal{G} q_n,$$
$$L_n(\theta) = \frac{1}{2} \mathbb{E}_{(s,a) \sim \rho}\left[ \left( q(s,a;\theta) - G^{\pi_n}_1(s,a,q_n) \right)^2 \right],$$
$$\theta_{n+1} \in \arg\min_{\theta} L_n(\theta),$$
$$q_{n+1}(s,a) = q(s,a;\theta_{n+1}).$$
</div>

Minimizing $L_n$ with a single step of stochastic approximation: Q-learning.  
Minimizing $L_n$ with SGD on a replay buffer (with a target network): DQN.

# Direct policy optimization

TODO: the PG, the REINFORCE PG, the PG theorem, the DPG theorem


# The Bellman equation as a linear program